In [ ]:
import pandas as pd
import os
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import fcluster 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn import preprocessing
import numpy as np
print(os.getcwd())
os.chdir('./../')
print(os.getcwd())

# Aprendizaje no supervisado

NO se dispone de una variable objetivo.

El objetivo es etiquetar a los datos con las características disponibles en el propio dataset.

## Algoritmos de agrupación o clustering
- K-means
- DBSCAN
- Jerárquico
- Muchos más: [Algoritmos implementados Scikit learn](https://scikit-learn.org/stable/modules/clustering.html)


## Funciones de comparación
- Distancia euclídea
- Distancia Manhattan
- Distancia de Chebychev

## Validación
Mucha complejidad en la validación de las agrupaciones.

Se necesita una validación del resultado haciendo uso de métricas y visualización.

# Objetivo en la práctica

1. Analizar la distribución de las estaciones buscando agrupaciones según su posición en la ciudad y la demanda que tienen en uso de bicicletas.
2. Aplicar técnicas de transformación de datos y búsqueda de parámetros óptimos para obtener agrupaciones de estaciones.


In [ ]:
df = pd.read_csv('./data/interim/estaciones.csv')
df.tail()
print(df.shape)
df.head()

In [ ]:
x_feat, y_feat = 'lon', 'lat'
fig, ax = plt.subplots(1, 1, figsize=(5, 4))
ax.scatter(df[x_feat], df[y_feat], s=2)
plt.ylabel(y_feat)
plt.xlabel(x_feat)
fig.tight_layout()
plt.show()

In [ ]:
lscol = ['uso_bici', 'lat', 'lon']
df.loc[:, lscol]
df.columns

In [ ]:
lscol = ['uso_bici', 'lat', 'lon']
min_max_scaler = preprocessing.MinMaxScaler()
minmx = min_max_scaler.fit(df.loc[:, lscol])
df2gm = pd.DataFrame(minmx.transform(df.loc[:, lscol]), columns=lscol)
df2gm

In [ ]:
x_feat, y_feat, size = 'lon', 'lat', 'uso_bici'
fig, ax = plt.subplots(1, 1, figsize=(5, 4))
ax = ax.scatter(df2gm[x_feat], df2gm[y_feat], s=(df2gm[size]+1)**8, c=df2gm[size], cmap='viridis')
plt.ylabel(y_feat)
plt.xlabel(x_feat)
fig.tight_layout()
cbar = plt.colorbar(ax)
cbar.set_label(size)
plt.show()

# Agrupación jerárquica
- Dendogramas, ordenación por distancias de las instancias y agrupación
- Calculo de matriz de similitud en función distancia
- Uso de variables numéricas
- No se define a priori un número de agrupaciones o k
- Se puede generar agrupaciones teniendo en cuenta diferentes parámetros
- Con el método single se pueden detectar outliers 

[Ejemplo visual de clustering](https://dashee87.github.io/data%20science/general/Clustering-with-Scikit-with-GIFs/)

[Explicación de clustering jerárquico](https://joernhees.de/blog/2015/08/26/scipy-hierarchical-clustering-and-dendrogram-tutorial/)



In [ ]:
def agrupacion_jerarquica(df, ls_cols, dist=3, method='ward',
                          metric='euclidean', optimal_ordering=False):
    mosaicstr="""
    ab
    ab
    cb
    """
    fig, ax = plt.subplot_mosaic(mosaic=mosaicstr, figsize=(10, 6))
    # relacion entre pares de instancias, distancia y acumulado
    # Metodo aglomerativo, no divisivo
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html
    Z = linkage(df.loc[:, ls_cols].values, method, 
                metric, optimal_ordering=optimal_ordering)
    dendrogram(Z, ax=ax['a'], color_threshold=dist)
    ax['a'].axhline(y=dist, color='r', linestyle='--')
    ax['a'].set_title(f"Dendograma - método {method}")
    ax['a'].set(xlabel='Estación', ylabel='Distancia')
    agglo_clusters = fcluster(Z, t=dist,criterion='distance')
    print("Nº Clusters: ", np.unique(agglo_clusters).shape[0])
    ax['b'].set_title(f"Clustering distribution k={np.unique(agglo_clusters).shape[0]}")
    ax['b'].set(xlabel=ls_cols[0], ylabel=ls_cols[1])
    axb = ax['b'].scatter(df[ls_cols[0]], df[ls_cols[1]], 
                          c=agglo_clusters, cmap='Dark2', s=6)
    cbar = plt.colorbar(axb)
    cbar.set_label('k-clusters')
    ax['c'].plot(Z[:, 2])
    ax['c'].axhline(y=dist, color='r', linestyle='--')
    # ax['c'].set_title(f"Distancia en la aglomeración")
    ax['c'].set(xlabel='Nº estaciones acumuladas', ylabel='Distancia')
    fig.tight_layout()
    plt.show()

Se pide buscar los mejores parámetros en:
- method: single, complete, average o ward
- metric: euclidean, mahalanobis o cityblock

method: https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html

metric: https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.pdist.html

In [ ]:
# Se busca ajustar por distancias la mejor agrupación
# metodos: single (minima distancia), complete (maxima distancia), average, ward
ag = agrupacion_jerarquica(df=df2gm, dist=0.2, ls_cols=['lon', 'lat'], 
                           method='single', metric='mahalanobis', 
                           optimal_ordering=True)

In [ ]:
agrupacion_jerarquica(df=df2gm, ls_cols=['lon', 'lat'], 
                      method='ward', 
                      dist=1, metric='euclidean', optimal_ordering=True)

In [ ]:
# Incluimos la variable de uso de bicis
agrupacion_jerarquica(df=df2gm, dist=0.1, 
                      ls_cols=['lon', 'lat', 'uso_bici'], 
                      method='single', metric='euclidean', optimal_ordering=True)

In [ ]:
# Comparamos sin aplicar escalado
agrupacion_jerarquica(df=df, dist=1000, ls_cols=['lon', 'lat', 'uso_bici'], 
                      method='ward', metric='euclidean', optimal_ordering=True)

# Kmeans
## Parametrización
- La inicialización es importante, pero la implementación en SKlearn
- Se define un número de agrupaciones objetivo antes del cálculo
- Es iterativo y tiene una complejidad de cáculo alta

## Evaluación
Existen diferentes métricas aplicables como en el clustering jerárquico:
- Coeficiente de silueta
- Davies-Bouldin
- Inercia o distorsión


In [ ]:
df2gm

In [ ]:
def plot_kmeans(df, max_k=10):
    """
    Plots KMeans clustering and the elbow method for determining the optimal k.

    Args:
        df (pd.DataFrame): DataFrame containing the data.
        max_k (int): Maximum number of clusters to test.
    """
    ls_cols = ['lon', 'lat', 'uso_bici']
    X = df.loc[:, ls_cols].values

    # Calculate inertia for different values of k
    inertia = []
    silhouette_scores = []
    for k in range(2, max_k + 1):
        kmeans = KMeans(n_clusters=k, random_state=22, n_init="auto").fit(X)
        inertia.append(kmeans.inertia_)
        silhouette_scores.append(silhouette_score(X, kmeans.labels_))

    # Plotting
    mosaicstr = """
    ab
    """
    fig, ax = plt.subplot_mosaic(mosaic=mosaicstr, figsize=(10, 5))

    # Plot KMeans clustering for the last k
    kn = max_k #using the max_k for the final plot.
    kmeans = KMeans(n_clusters=kn, random_state=22, n_init="auto").fit(X)
    labels1 = kmeans.labels_
    centroids1 = kmeans.cluster_centers_

    ax['b'].set_title(f"Clustering distribution k={kn}")
    ax['b'].set(xlabel=ls_cols[0], ylabel=ls_cols[1])
    axb = ax['b'].scatter(X[:, 0], X[:, 1], c=labels1, cmap='Dark2', s=10)
    cbar = plt.colorbar(axb)
    cbar.set_label('k-clusters')
    ax['b'].scatter(centroids1[:, 0], centroids1[:, 1], marker='x',
                    s=200, c='black')

    # Codo
    ax['a'].plot(range(2, max_k + 1), inertia, marker='o')
    ax['a'].set_title('Elbow Method')
    ax['a'].set_xlabel('Number of clusters (k)')
    ax['a'].set_ylabel('Inertia')

    fig.tight_layout()
    plt.show()
    return kmeans.get_params()


In [ ]:
params = plot_kmeans(df2gm, 2)

In [ ]:
ls_cols = ['lon', 'lat', 'uso_bici']
X = df2gm.loc[:, ls_cols].values
params['n_clusters'] = 15
kmeans = KMeans(**params).fit(X)
silhouette_score(X, kmeans.labels_), kmeans.inertia_

# Conclusiones
- Se prueba el uso de transformaciones para obtener conocimiento 
- Se obtiene agrupaciones que puen ajustarse mejor a las delimitaciones de barrio